# Fashion review generator

In this project, I am going to use product reviews written by genuine shoppers and generate new product reviews copy. The purpose is to demonstrate the usefulness of RNN and whether they can be used to create advertising copy for ecommerce. The data set comes from:
https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews (requires Kaggle login).

This is a genuine product review: 

>_Snap this one up but make sure you order a size larger than your normal size. it runs a full size too small. it sold out before and i was told it wasn't coming back in. i had purchased one previously in my regular size but it was too small. to my delight, it's back, and i've purchased one in the next size up. it's well made, can go from business to sport to casual._

This is a review generated by our RNN: 

>_this is the right color and i am a little small. the fabric is soft and the dress was very pretty and the fabric looks great on me._

To make the fashion review generator, we take the following steps:
- Get and load the data.
- Create a vocabulary look up table based on the complete set of reviews. This transforms words into integers (which our RNN can be trained on), and a reverse table to turn integers back into words.
- Tokenize punctuation which removes duplicates (e.g. good and good!).  
- Pre-process the product reviews by creating a vocabulary look up table, replacing the punctuation with tokens, converting text to lower case, split them up and turning each word into an integer from the look up table.
- Batching the data ready for training.
- Build the recurrent neural network / LTSM.
- Define forward pass and back propagation.
- Train the RNN with hyper parameter tuning to achieve desired level of performance.
- Save check point.
- Generate new reviews by supplying a small number of words. 

### Load libraries
Loading Python libraries required for our processing the data, defining and training our RNN.

In [1]:
# Load file operation libraries
import os
import pickle
import csv

# Load data manipulation libraries
import numpy as np
from collections import Counter

# Load PyTorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

### Check availability of GPU
RNN takes ages to train on a CPU. So it is best to train our network on a GPU enabled machine. 

In [2]:
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print('Training on GPU.')
else:
    print('No GPU found: using CPU instead.')

Training on GPU.


## Get the data

We download and unzip the data file from Kaggle and place it in the `data` folder. The CSV file `Womens Clothing E-Commerce Reviews.csv` is in the data directory. We use `csv` to read the reviews and store in `fashion_reviews` for processing later. Now `fashion_reviews` only contains a list of reviews, their rating (5 point scale) and the recommendation (1 = recommend, 0 = not recommended). Since we are only interested in the reviews where a purchase is recommended by the shopper, we discard the non-recommendations.

In [3]:
# Read the CSV file and only import the columns containing the review copy, 
# 5 point scale and recommendation

fashion_reviews = []
with open('data/Womens Clothing E-Commerce Reviews.csv') as csvfile:
    review_reader = csv.reader(csvfile, delimiter = ',', quotechar = '"')
    for row in review_reader:
        fashion_reviews.append(row[4:7])
    # Remove the header (first) row which we won't use
    fashion_reviews.pop(0)

# Store "recommended" reviews in recommended_reviews
recommended_reviews = []
for row in fashion_reviews:
    if row[2] == '1':
        recommended_reviews.append(row[0])

## Explore the data

Taking a look at how many reviews there are, and print out a random sample of the reviews. 


In [5]:
# Determine number of reviews
n_reviews = len(fashion_reviews)
n_recommended_reviews = len(recommended_reviews)
print('There are {} reviews in total, of which {} recommend a particular product.'.format(n_reviews,
                                                                                          n_recommended_reviews))

# Calculate average number of words per review
word_count_row = [len(row.split()) for row in recommended_reviews]
print('Average number of words in each recommended review is', np.average(word_count_row))

# Show a sample selection of reviews
print("\nHere is a sample of reviews:")
for idx in range(5):
    i_review = np.random.randint(0, n_recommended_reviews)
    print("\nReview {} is: {}".format(i_review, recommended_reviews[i_review]))

There are 23486 reviews in total, of which 19314 recommend a particular product.
Average number of words in each recommended review is 57.40100445272859

Here is a sample of reviews:

Review 13978 is: The dress is beautiful, i wanted to like it and could hardly wait to reveive it. it fits beautifully, and snug except around the arms where it is huge. i could only wear it with a tshirt underneat.

Review 13637 is: This dress is absolutely gorgeous! the top portion fit a little smaller than the typical size i order. the zip side was a little snug, but the fit is very flattering! the lace detail and belted design are very pretty. the skirt portion has more volume than i tend to like, but it is it beautiful!

Review 18835 is: I ordered this online and i love the style, but didn't like the print as much as i thought i would.

Review 11064 is: I love this dress. the fabric is butter soft, the strapless top stays in place and it hangs beautifully. it is such a wonderfully comfortable summer d

## Create pre-processing functions

Since a neural network cannot "read" the text in the reviews, we need to convert the text into integers that our neural network can learn from. We first create a lookup table for all the words in the reviews, then we create a reverse lookup table for when we can generate new reviews. Punctuations form an integral part of any reviews. Just like words, a neural network cannot read them. Here we will replace punctuations in the reviews with a token. 

Unlike sentiment analysis, we are including all stop words (e.g. 'to', 'the') and do not make a distinction of words that have the same stem (e.g. 'beauty', 'beautiful', 'beatifully').

In [6]:
# This function creates a lookup table of all the words used in the review. 
def create_lookup_tables(reviews):
    # Use Counter to count the words
    counted_words = Counter(reviews)
    
    # Sort the words in decending order of occurance
    sorted_vocab = sorted(counted_words, key = counted_words.get, reverse = True)

    # Create int_to_vocab dictinoaries
    int_to_vocab = {idx: word for idx, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: idx for idx, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab

In [9]:
# This function returns a dictionary of punctuation tokens that will be used to 
# replace punctuations in reviews with a token. 
def token_lookup():    
    tokens_dict = {'.':'<FULLSTOP>',
                   ',':'<COMMA>',
                   '?':'<QUESTION_MARK>',
                   '!':'<EXCLAMATION_MARK>',
                   '"':'<QUOTATION_MARK>',
                   ';':'<SEMICOLON>',
                   '-':'<DASH>',
                   '(':'<LEFT_PAREN>',
                   ')':'<RIGHT_PAREN>',
                   '\n':'<CARRIAGE_RETURN>'}
    return tokens_dict

In [10]:
# These two functions saves the pre processed data, and load them
# back at a subsequent session.

padding_words = {'PADDING': '<PAD>'}

def preprocess_and_save_data(reviews, token_lookup, create_lookup_tables, filename):
    # The reviews are in a list, and they need to be converted into 
    # a single piece of text.
    reviews = "".join(reviews)
    
    # Create the punctuation token lookup and replace all punctuations
    # in the reviews with them.
    token_dict = token_lookup()
    for key, token in token_dict.items():
        reviews = reviews.replace(key, ' {} '.format(token)) 
    # The additional spaces are essential to separate punctuations from words

    # Covert allwords to lower case, then split them into individual
    # words to form a tuple.
    reviews = reviews.lower()
    reviews = reviews.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(reviews + list(padding_words.values()))
    # int_review contains integer-ized version of the aggregated reviews
    int_reviews = [vocab_to_int[word] for word in reviews]
    
    pickle.dump((int_reviews, vocab_to_int, int_to_vocab, token_dict), 
                open(filename, 'wb'))
    
def load_preprocess(filename):
    # Load the preprocessed data (just saved) and return them in batches
    return pickle.load(open(filename, mode='rb'))


In [11]:
# Now running the pre-processing and save the data 
preprocess_filename = 'preprocess.p'
preprocess_and_save_data(recommended_reviews, token_lookup, create_lookup_tables, preprocess_filename)


### Load processed data and check point ###

We call the `load_preprocess` function to load the pre-processed data. Use this function to re-load the pre-processed data without re-processing the data. 

In [27]:
### CHECK POINT ###
padding_words = {'PADDING': '<PAD>'} # this value is used later

preprocess_filename = 'preprocess.p'
int_reviews, vocab_to_int, int_to_vocab, token_dict = load_preprocess(preprocess_filename)

print(int_reviews[:50])

[254, 491, 19, 876, 3, 613, 3, 6341, 8, 20, 11, 35, 1812, 105, 0, 2, 1623, 9, 208, 6, 10, 5, 106, 4, 3, 40, 287, 2, 145, 1336, 2, 425, 53, 27, 64, 6, 149, 1336, 35, 98, 0, 2, 72, 5, 98, 3, 41, 509, 39, 0]


In [34]:
print(vocab_to_int['dress'])

20


## Batch the data for training and define the model

### Batch the data for training

Our RNN will train on data in batches. The `batch_data` function below will take in a sequnece of pre-processed words and batch them into pre-specified lengths. Batching also defines the length of a sequence of words (`sequence_length`) and the target word.

For example, if we have a sentence 
>this is a very nice dress and the color suits me.

and `sequence_length = 5`, then

Features: [8, 7, 5, 26, 75] for 'this is a very nice'

Target: 20 for 'dress'

This function will go through the tokenized review provided, and turn them into batches of features and target tensors using PyTorch's `DataLoader`.

In [35]:
def batch_data(words, sequence_length, batch_size):
    # Define number of batches
    n_batches = len(words)//batch_size
    # ... and discard orphan words of incomplete batches
    words = words[:n_batches * batch_size]
    # print('words:', words)
    
    # Define feature_tensors and target_tensors 
    feature_tensors, target_tensors = [], []
    
    # Going through words in the reviews, one batch (batch_size as step) at a time
    # For each batch of the total number of batches
    for idx_batch in range(0, len(words), batch_size):
    
        this_batch = words[idx_batch : idx_batch + batch_size + sequence_length]
        for idx_word in range(batch_size):
            if batch_size - idx_word >= sequence_length + 1:                
                # get feature
                batch_feature = this_batch[idx_word:idx_word + sequence_length]
                # print('{} batch_feature: {}'.format(idx_word, batch_feature))
                feature_tensors.append(batch_feature)
                
                # get target which is the following (sequence_length) set of words
                batch_target = this_batch[idx_word + sequence_length]
                # print('{} batch_target: {}'.format(idx_word + sequence_length, batch_target))
                target_tensors.append(batch_target)
                
            else:
                pass

    feature_tensors = torch.from_numpy(np.asarray(feature_tensors))
    feature_tensors = torch.LongTensor(feature_tensors)
    print('\nfeature_tensors size:', feature_tensors.size())
    
    target_tensors = torch.LongTensor(target_tensors) # a LongTensor is expected in the loss function
    print('\ntarget_tensors size:', target_tensors.size())
    
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle = True )

    return data_loader 


### Define the recurrent neural network (LSTM)

We are implementing a RNN using PyTorch's `nn.Module` class, which requires us to specify:
- number of embedding features
- number of hidden features
- number of layers in the LSTM network 
- and whether the input and output tensors are batched

The feed forward function passes the batched input through the LSTM layers (with dropout) and onto a fully connected layer.

We also define a function to initiate the hidden features as zeroes. 

In [36]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.n_embedding = embedding_dim 
        self.n_hidden = hidden_dim
        self.n_layers = n_layers
        self.drop_prob = dropout
        
        # define model layers
        self.embedding = nn.Embedding(self.vocab_size, self.n_embedding)
        
        self.lstm = nn.LSTM(self.n_embedding,
                            self.n_hidden,
                            self.n_layers,
                            batch_first = True)
        self.dropout = nn.Dropout(self.drop_prob)
        self.fc = nn.Linear(self.n_hidden, self.vocab_size)
    
    def forward(self, nn_input, hidden):
        batch_size = nn_input.size(0)
        # Put input into embedded layer
        embed = self.embedding(nn_input) 
        # Then onto LSTM layers
        output, hidden = self.lstm(embed, hidden)
        output = output.contiguous().view(-1, self.n_hidden) # stacking the LSTM
        output = self.dropout(output)
        # Then onto output layer
        output = self.fc(output)

        output = output.view(batch_size, -1, self.output_size) 
        output = output[:, -1] # get last batch

        # return one batch of output word scores and the hidden state
        return output, hidden
    
    def init_hidden(self, batch_size):
        # Initialize the hidden state of an LSTM with zero weights,
        # and move to GPU if available
        
        weight = next(self.parameters()).data
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        return hidden

### Define the forward and backpropagation functions


In [37]:
def forward_back_prop(rnn, optimizer, criterion, input_data, target, hidden):
    
    if train_on_gpu:
        input_data, target = input_data.cuda(), target.cuda()
    
    hidden = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    # Perform forward pass
    output, hidden = rnn(input_data, hidden)
    
    # Perform backpropagation
    loss = criterion(output, target)
    loss.backward()
    
    # Avoid gradient explosion through clipping
    nn.utils.clip_grad_norm_(rnn.parameters(), max_norm = 5) # max_norm = 5 seems to be the convention
    optimizer.step()    
    
    # Calculate loss
    this_loss = loss.item()

    # Return the loss over a batch and the hidden state produced by our model
    return this_loss, hidden


### Defining the training function

In [38]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    
    batch_losses = []
    
    # Putting the model in training mode
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # This ensures we iterate over completely full batches only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # Feed forward then backpropagation
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # Record loss
            batch_losses.append(loss)

            # Print loss stats as we train
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained model
    return rnn

### Setting upt the data loader and training parameters

The `DataLoader` will use the specified sequence_length and batch size. 

In [93]:
# Setting up the data loader for training
sequence_length = 25 # Setting to a sequence length of 25 (roughly the length of a sentence)
batch_size = 100 # Batch Size
train_loader = batch_data(int_reviews, sequence_length, batch_size)



feature_tensors size: torch.Size([951375, 25])

target_tensors size: torch.Size([951375])


### As for the training parameters, we use the following:
- Epochs 3, which will help us reach a loss of about 3.75, sufficient for our purpose.
- Learn rate is 0.001 which is commonly recommended in LSTM literature
- Vocabluary size is already calculated while pre-processing the data, and output size is the same.
- Embedding dimension and hidden dimension are set to a fraction of the vocabulary size, so that the network complexity reflects the number of unique words it has to deal with. 

In [94]:
# Setting up taining parameters
num_epochs = 3
learning_rate = 0.001
vocab_size = len(vocab_to_int) # Vocab size
output_size = vocab_size # Output size
fraction = 0.025

# Embedding dimension
embedding_dim = int(vocab_size * fraction) # setting this to 5% of the vocab size
print('vocab_size is {}, embedding_dim is {} by setting latter to {} of former.'.format(vocab_size,
                                                                                        embedding_dim,
                                                                                        fraction))
# Hidden dimension
hidden_dim = int(vocab_size * fraction)
print('vocab_size is {}, hidden_dim is {} by setting latter to {} of former.'.format(vocab_size,
                                                                                        embedding_dim,
                                                                                        fraction))

# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

vocab_size is 17487, embedding_dim is 437 by setting latter to 0.025 of former.
vocab_size is 17487, hidden_dim is 437 by setting latter to 0.025 of former.


### Start training
We now start training our network with the pre-processed data (already in `DataLoader`) by specifying the optimizer (in this case Adam) and criterion (CrossEntrophyLoss). We are aiming to achieve loss of around 3.5.

In [ ]:
# Create model and move to GPU if available
this_RNN = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)

if train_on_gpu:
    this_RNN.cuda()

# Defining loss and optimization functions for training
optimizer = torch.optim.Adam(this_RNN.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training the model
trained_RNN = train_rnn(this_RNN, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)


Training for 3 epoch(s)...
Epoch:    1/3     Loss: 5.890889410972595
Epoch:    1/3     Loss: 5.187287108421326
Epoch:    1/3     Loss: 4.97631830072403
Epoch:    1/3     Loss: 4.820603412628174
Epoch:    1/3     Loss: 4.743888828754425
Epoch:    1/3     Loss: 4.66372947883606
Epoch:    1/3     Loss: 4.597059538364411
Epoch:    1/3     Loss: 4.529038254737854
Epoch:    1/3     Loss: 4.485850398540497
Epoch:    1/3     Loss: 4.443445352077484
Epoch:    1/3     Loss: 4.4339104485511776
Epoch:    1/3     Loss: 4.392889785766601
Epoch:    1/3     Loss: 4.3864826965332036
Epoch:    1/3     Loss: 4.362711019992829
Epoch:    1/3     Loss: 4.3401008892059325
Epoch:    1/3     Loss: 4.313764045238495
Epoch:    1/3     Loss: 4.283340344905853
Epoch:    1/3     Loss: 4.274296568393707
Epoch:    1/3     Loss: 4.240500148773194
Epoch:    2/3     Loss: 4.138813294397693
Epoch:    2/3     Loss: 4.1313345084190365
Epoch:    2/3     Loss: 4.126631410598755
Epoch:    2/3     Loss: 4.132560437679291
Epoch

A loss of less than 4 is sufficient for the purpose of generating fashion reviews, as we will see. 

## Generate fashion reviews 

We now want to use our trained RNN to generate fashion reviews. The following function takes in the trained model, a "start word", our pre-processed data and the number of words to predict to produce a new review. 

In [96]:
def generate(rnn, start_word_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    
    # Put the network in evaluation mode
    rnn.eval()
    
    # Create an empty sequence with pad_value and batch size 1
    current_seq = np.full((1, sequence_length), pad_value)
    
    # The predicted sequence starts with the start_word (!)
    current_seq[-1][-1] = start_word_id
    predicted = [int_to_vocab[start_word_id]]

    for _ in range(predict_len):

        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # Initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # Get the output of the rnn by providing a sequence
        output, _ = rnn(current_seq, hidden)
        
        # Get the next word probabilities using Softmax
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu 
         
        # Use top_k sampling to get the index of the next  
        # most probable 10 words
        top_k = 10
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # Pick one of these five words randomly as the next word
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # Retrieve that word from the dictionary 
        word = int_to_vocab[word_i]
        predicted.append(word)    
        
        # Move tensor back to CPU for Numpy
        current_seq = current_seq.to(torch.device("cpu"))
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    generated_review = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        generated_review = generated_review.replace(' ' + token.lower(), key)
    generated_review = generated_review.replace('\n ', '\n')
    generated_review = generated_review.replace('( ', '(')
    
    # Return generated review
    return generated_review

In [101]:
def generate_new_review(model):
    
    start_word = input('Enter start word:') # is the word at the beginning of the script
    start_word = start_word.lower() # as the vocab_to_int is in all lower case

    review_length = int(input('Enter the length of a new review (number of words):')) # this specifies the number of words in the generated review
    
    # Generating 3 random reviews
    print('\nGenerating random reviews:')
    for idx in range(3):
        new_review = generate(model, vocab_to_int[start_word], 
                          int_to_vocab, token_dict, vocab_to_int[padding_words['PADDING']], review_length)
        print('Generated review:\n', new_review, '\n')


In [103]:
generate_new_review(trained_RNN)

Enter start word:the
Enter the length of a new review (number of words):75

Generating random reviews:
Generated review:
 the length, i bought both colors. it's a great dress but i love the fit in the front, which is a little more relaxed. the colors are beautiful and the fabric is a lovely and soft detail. i bought this dress today and it was great. this top is super comfortable and comfortable and it is a great piece for fall, and the fabric is a great addition to 

Generated review:
 the color. i got the pink and it has a nice touch of colors. it's not a dress for a wedding. i love the fabric and the fit, but i didn't find the skirt for a more comfortable fit. i was swimming in this top so much that you don't find the dress too tight in it to have a small, but i didn't need the small to be a 

Generated review:
 the fabric and the fit. i am 5" and it fits well. i am usually a 2 but bought the small, and i ordered this in my usual size s which fits great on my frame. the fabric is li

### Saving  the model
We are saving the model so that we can come back at a later date for future inference. This involves saving:
- model architechture parameters 
- parameters of the trained model

In [104]:
# Saving model parameters and state_dict
save_checkpoint ={'vocab_size': vocab_size,
                  'output_size': output_size,
                  'embedding_dim': embedding_dim,
                  'hidden_dim': hidden_dim,
                  'n_layers': n_layers,
                  'dropout': 0.5,
                  'learning_rate': learning_rate,
                  'model_state_dict': trained_RNN.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict()
                 }

torch.save(save_checkpoint, 'trained_fashion.pt')

### Loading the model and setting parameters
We can re-load this saved model for futher "play" with generating fashion reviews. We need to ensure the following parameters are set.

In [105]:
# Load pre-processed vocabularty
preprocess_filename = 'preprocess.p'
int_reviews, vocab_to_int, int_to_vocab, token_dict = load_preprocess(preprocess_filename)

padding_words = {'PADDING': '<PAD>'}

sequence_length = 25 # Setting to a sequence length of 25 (roughly the length of a sentence)


We also need to run the following further up in this Notebook to ensure our saved model will load and run properly. 
- class RNN 
- def generate
- def generate_new_review

In [106]:
# Loading model parameters and state_dicts
load_checkpoint = torch.load('trained_fashion.pt')

# Re-create model
loaded_rnn =  RNN(load_checkpoint['vocab_size'],
                  load_checkpoint['output_size'],
                  load_checkpoint['embedding_dim'],
                  load_checkpoint['hidden_dim'],
                  load_checkpoint['n_layers'],
                  load_checkpoint['dropout'])

# Load model state_dict
loaded_rnn.load_state_dict(load_checkpoint['model_state_dict'])

# Specify optimizer and state_dict
optimizer = torch.optim.Adam(loaded_rnn.parameters(), lr=load_checkpoint['learning_rate'])
optimizer.load_state_dict(load_checkpoint['optimizer_state_dict'])

# Move model to GPU
if train_on_gpu:
    loaded_rnn.cuda()

loaded_rnn.eval()

RNN(
  (embedding): Embedding(17487, 437)
  (lstm): LSTM(437, 437, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=437, out_features=17487, bias=True)
)

In [107]:
generate_new_review(loaded_rnn)

Enter start word:that
Enter the length of a new review (number of words):100

Generating random reviews:
Generated review:
 that i would have bought the white, and i love the style. the fabric is beautiful and the material is very soft but it makes it perfect. the fabric is very soft and a little more like it is not the perfect sweater. i ordered a medium but it fits just right. i am 5'8" and i have a petite chest, which is so i have broad shoulders and this dress looks perfect. it is so comfortable but looks great.
the fit is great on me! it is a great 

Generated review:
 that i have a larger bust. i have a short torso and the xs was perfect!

the color is very soft, and it is very cute and a little too tight. it's so pretty. i got the pink, and it looked a tad more versatile. i think you can pair it with a jacket or dress or dress for work.
- i would have purchased it online, but i am not sure if i had to get a small(and i got it in the red(and 

Generated review:
 that i was worrie